 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P TSX 60 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp_tsx_history.csv`: Contains historical closing prices of the S&P TSX 60 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data.

In [ ]:
# Reading whale returns

whale_returns = Path("./Resources/whale_returns.csv")
whale_returns = pd.read_csv(whale_returns, index_col="Date", infer_datetime_format=True, parse_dates=True)

whale_returns.head()

In [ ]:
# Count nulls
whale_returns.isnull().sum()

In [ ]:
# Drop nulls
whale_returns_df = whale_returns.dropna()
whale_returns_df


## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data.

In [ ]:
# Reading algorithmic returns
algo_returns_data = Path("./Resources/algo_returns.csv")
algo_returns = pd.read_csv(algo_returns_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
algo_returns.head()

In [ ]:
algo_returns.dtypes

In [ ]:
# Count nulls
algo_returns.isnull().sum()

In [ ]:
# Drop nulls
algo_returns_df = algo_returns.dropna()
algo_returns_df.head()

## S&P TSX 60 Returns

Read the S&P TSX 60 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P TSX 60 Closing Prices
sp_tsx_history_data = Path("./Resources/sp_tsx_history.csv")
sp_tsx_history_df = pd.read_csv(sp_tsx_history_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp_tsx_history_df.head()

In [ ]:
# Check Data Types
sp_tsx_history_df.index

In [ ]:
# Fix Data Types

#Drop $symbol 

sp_tsx_history_df["Close"] = sp_tsx_history_df["Close"].str.replace("$", "").str.replace("," , "") #.str acces values

# sp_tsx_history_df["Close"] = pd.to_numeric(sp_tsx_history_df["Close"])
sp_tsx_history_df


In [ ]:
sp_tsx_history_df.dtypes

In [ ]:
#converted the object into an int so i can run formulas
sp_tsx_history_df["Close"] = pd.to_numeric(sp_tsx_history_df["Close"])
sp_tsx_history_df

In [ ]:
# Calculate Daily Returns
daily_returns = sp_tsx_history_df.pct_change()
daily_returns

In [ ]:
# Drop nulls
daily_returns = daily_returns.dropna()
daily_returns

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
daily_returns.rename(columns={'Close': 'SP TSX Daily Return'}, inplace=True)
daily_returns.tail()


In [ ]:
sp_tsx_daily_returns = daily_returns.dropna()
sp_tsx_daily_returns.head()

## Combine Whale, Algorithmic, and S&P TSX 60 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P TSX 60 Returns into a single DataFrame with columns for each portfolio's returns.

combined_df = pd.concat([whale_returns_df,algo_returns_df,sp_tsx_daily_returns], axis=1, join='inner')

combined_df

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios

combined_df.plot()


#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios

cumlative_combined_df = (1 + combined_df).cumprod()


# Plot cumulative returns
cumlative_combined_df.plot(title="my title")

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios.
4. Determine which portfolios are riskier than the S&P TSX 60.
5. Calculate the Annualized Standard Deviation.

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk

cumlative_combined_df.boxplot(rot=45, figsize=(15,10))

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
cumlative_combined_df.std()

### Determine which portfolios are riskier than the S&P TSX 60

In [ ]:
# Calculate  the daily standard deviation of S&P TSX 60

daily_returns.std()
sp_risk = cumlative_combined_df['SP TSX Daily Return'].std()


# Determine which portfolios are riskier than the S&P TSX 60

cumlative_combined_df.std() > sp_risk

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
cumlative_combined_df.std() * np.sqrt(252)

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window.
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P TSX 60.
3. Choose one portfolio, then calculate and plot the 60-day rolling beta for it and the S&P TSX 60.

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolling_std = cumlative_combined_df.rolling(window=21).std()


# Plot the rolling standard deviation
cumlative_combined_df.rolling(window=21).std().plot(rot=45, figsize=(15,10))

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
correlation = cumlative_combined_df.corr()
# Display de correlation matrix *Go back to fix
correlation.corr().plot(rot=45, figsize=(20,10))

In [ ]:
cumlative_combined_df.head()

### Calculate and Plot Beta for a chosen portfolio and the S&P 60 TSX

In [ ]:
# Calculate covariance of a single portfolio

risk_free_Rate = 0.04 #assuming from the internet

covariance_algo_1 = cumlative_combined_df['Algo 1'].cov(daily_returns['SP TSX Daily Return'])
covariance_algo_1

# Calculate variance of S&P TSX
variance_SP = daily_returns['SP TSX Daily Return'].var()
variance_SP

# Computing beta

algo_1_beta = covariance_algo_1 / variance_SP
algo_1_beta

# Plot beta trend
#algo_1_beta.plot(figsize=(20, 10))


## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window df.ewms()

rolling_window_ewm = cumlative_combined_df.ewm(21).std()
rolling_window_ewms

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios

# Calculate Sharpe Ratio
sharpe_ratios = ((cumlative_combined_df.mean()-cumlative_combined_df['SP TSX Daily Return'].mean()) * 252) / (cumlative_combined_df.std() * np.sqrt(252))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind="bar", title="Sharpe Ratios")

### Determine whether the algorithmic strategies outperform both the market (S&P TSX 60) and the whales portfolios.

Aglo 1 outperform both markets

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P TSX 60. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock.
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns.
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others.
5. Include correlation analysis to determine which stocks (if any) are correlated.

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock

# Reading whale returns

l_returns = Path("./Resources/l_historical.csv")
l_returns = pd.read_csv(l_returns, index_col="Date", infer_datetime_format=True, parse_dates=True)
l_returns.rename(columns = {'Close':"l"}, inplace = True)
l_returns.drop(columns= 'Symbol', inplace = True)

l_returns.head()

#pick 3 - pd.concat pick dates from 2015-2019

In [ ]:
# Reading data from 2nd stock
otex_returns = Path("./Resources/otex_historical.csv")
otex_returns = pd.read_csv(otex_returns, index_col="Date", infer_datetime_format=True, parse_dates=True)
otex_returns.rename(columns = {'Close':"otex"}, inplace = True)
otex_returns.drop(columns= 'Symbol', inplace = True)

otex_returns.head()

In [ ]:
# Reading data from 3rd stock

shop_returns = Path("./Resources/shop_historical.csv")
shop_returns = pd.read_csv(shop_returns, index_col="Date", infer_datetime_format=True, parse_dates=True)
shop_returns.rename(columns = {'Close':"shop"}, inplace = True)
shop_returns.drop(columns= 'Symbol', inplace = True)

shop_returns.head()


In [ ]:
# Combine all stocks in a single DataFrame
combined_df = pd.concat([l_returns, otex_returns ,shop_returns], axis= 'columns', join='inner')

combined_df.head()

In [ ]:
# Reorganize portfolio data by having a column per symbol

daily_returns = combined_df.pctchange()


In [ ]:
# Calculate daily returns
daily_returns = combined_df.pct_change()


# Drop NAs
daily_returns = daily_returns.dropna()


# Display sample data
daily_returns.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]
l_weight = 1/3
otex_weight = 1/3
shop_weight = 2/3

# Calculate portfolio return

protfolio_return = (l_weight * combined_df["l"] + otex_weight * combined_df["otex"] + shop_weight * combined_df["shop"])

# Display sample data
protfolio_return.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame

all_df = pd.concat([whale_returns_df,algo_returns_df,sp_tsx_daily_returns,l_returns, otex_returns ,shop_returns], axis= 'columns', join='inner')
all_df

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
all_df=all_df.dropna()

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`

all_daily_return = all_df.pct_change()
daily_std = all_daily_return.std()


annualized_std = daily_std * np.sqrt(252)
annualized_std.dropna()


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
all_df.rolling(window=21).std()
# Plot rolling standard deviation
all_df.rolling(window=21).std().plot( title = "Rolling Standard Deviation", figsize = (20,10))

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
correlation_all = all_df.corr()
correlation_all.plot(title = "Correlation", figsize = (20,10))
correlation_all


### Calculate and Plot the 60-day Rolling Beta for Your Portfolio compared to the S&P 60 TSX

In [ ]:
# Calculate and plot Beta
covariance_all_daily_return = all_daily_return

beta = covar/variance


In [ ]:
covar = all_daily_return.mean().std() * np.sqrt(252)

covar

variance = all_daily_return.var()


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios

all_daily_return.std() * np.sqrt(252)
all_daily_return

In [ ]:
# Visualize the sharpe ratios as a bar plot

.plot(kind='bar')

### How does your portfolio do?

Write your answer here!

# Amazing !